In [2]:
from Bio import SeqIO
import os
import re
import pandas as pd
from functools import reduce
from shutil import copyfile
#import community
import pickle

In [3]:
pattern=re.compile('\((.*?)\)')

In [4]:
def check(secu,tipo):
    for i in secu.features:
        if i.type=='cluster':
            if i.qualifiers['product'][0]==tipo:
                return True
            else:
                return False
            break

In [5]:
yo=pd.read_csv('v2_nodes_after_cleaning.csv')

In [6]:
yo2=yo.copy()

In [7]:
ty=[]
for w in yo2['Id']:
    boo=SeqIO.read('clusters/'+w+'.gbk','genbank')
    ty.append(len(boo))

In [9]:
yo3=yo2.copy()
yo3['length']=ty

In [10]:
yo3=yo3[['Id','length']].sort_values('length')

In [14]:
clu=[]
leng=[]
cds=[]
bio_cds=[]
counter1=0
counter2=0
#amp=[]
val_amp=0
for i,l in zip(yo3['Id'],yo3['length']):
    q=SeqIO.read('clusters/'+i+'.gbk','genbank')
    sumario_cds=[cat.qualifiers for cat in q.features if (cat.type=='CDS')]
    o_sec=[j for j in sumario_cds if ('sec_met' in j.keys())]
    nc=[]
    for a in o_sec:
        tipox=a['sec_met'][0].split(':')[1].strip()
        try:
            kind=a['sec_met'][2].split(':')[1].strip()
        except:
            kind='none'
        if check(q,'nrps'):
            if (kind=='biosynthetic')&(tipox=='nrps'):
                nc.append(a)
        else:
            if kind=='biosynthetic':
                nc.append(a)
    sumario=[cat.qualifiers for cat in q.features if (cat.type=='PFAM_domain')]
    f=lambda x:x['domain'][0]
    if (len(sumario)!=0)&(check(q,'nrps')):
        dominios=list(map(f,sumario))
        val_amp=dominios.count('AMP-binding')
    #if (len(sumario_cds)>=2) & (len(nc)>=1) & (len(sumario_cds)>len(nc)) & (not(check(q,'nrps'))):
    #if (check(q,'nrps'))&(len(sumario_cds)>=2) & (len(nc)>=1) & (len(sumario_cds)>len(nc)):
        #counter1+=1
    #if (check(q,'nrps'))&(len(sumario_cds)>=2) & (len(nc)>=1) & (len(sumario_cds)>len(nc)) & (val_amp>=2):
        #counter2+=1
    if (check(q,'nrps'))&(len(sumario_cds)>=2) & (len(nc)>=1) & (len(sumario_cds)>len(nc)) & (val_amp>=2):
        clu.append(i)
        leng.append(l)
        cds.append(len(sumario_cds))
        bio_cds.append(len(nc))
        #amp.append(val_amp)
    if (len(sumario_cds)>=2) & (len(nc)>=1) & (len(sumario_cds)>len(nc)) & (not(check(q,'nrps'))):
        clu.append(i)
        leng.append(l)
        cds.append(len(sumario_cds))
        bio_cds.append(len(nc))

In [18]:
counter2

718

In [10]:
tempo=pd.DataFrame({'Id':clu,'length':leng,'num_cds':cds,'num_bio_cds':bio_cds}) #,'amp_binding':amp

In [12]:
tu1=yo.copy()
spe=yo[['Id','reduccion']]

In [13]:
tu1.set_index('Id',inplace=True)

In [14]:
selec=tu1.loc[tempo['Id'].tolist(),:]

In [15]:
selec2=selec.reset_index('Id')

In [16]:
with open('rhodo_contig_genome_dict.pkl','rb') as f:
    values=pickle.load(f)

In [17]:
tre=selec2['Id'].apply(lambda x:x[0:-11].split('.')[0]+'.1').apply(lambda x:values[x])

In [18]:
len(pd.unique(tre))

110

In [19]:
yo['reduccion'].value_counts()

NRPS             988
Other            365
Terpene          194
PKS              193
RiPPs            130
Ectoine          112
Butyrolactone    108
Other hybrids    102
Name: reduccion, dtype: int64

In [20]:
#selec2[~selec2['Label'].isna()]
selec2['reduccion'].value_counts()

NRPS             718
Other            340
Terpene          194
PKS              192
RiPPs            130
Ectoine          112
Butyrolactone    108
Other hybrids    102
Name: reduccion, dtype: int64

In [19]:
#selec2['reduccion'].value_counts().to_excel('conteo_154genomas+BGC_check_control.xlsx')

In [22]:
selec2.to_csv('v2_nodes_after_cleaning_checkcontrol.csv',index=False)

In [23]:
chosen=selec2['Id'].tolist()

In [24]:
tempo2=tempo.merge(spe,how='left',on='Id')

In [25]:
tempo2=tempo2.sort_values('length')

In [26]:
tempo2.to_csv('v2_info_nodes_after_cleaning_checkcontrol.csv',index=False)

In [27]:
edgo=pd.read_csv('v2_edges_06_after_cleaning.csv')

In [28]:
edgonuevo=edgo.loc[(edgo['Source'].isin(chosen))&(edgo['Target'].isin(chosen))]

In [29]:
edgonuevo.to_csv('v2_edges_06_after_cleaning_checkcontrol.csv',index=False)